In [1]:
!pip install --upgrade mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.1 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import os
from glob import glob
import zipfile
import shutil
import cv2
import random

import re
import zipfile
import json

from google.colab import drive

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import matplotlib.colors as mcl

from matplotlib.colors import LinearSegmentedColormap
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [3]:
# data_path = "/content/GoogleDrive/MyDrive/Colab Notebooks"
data_path = "/content/GoogleDrive/MyDrive"
ROOT_DIR = "/content"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
drive.mount("/content/GoogleDrive/")

# shutil.copy(os.path.join(data_path,"test1801_9999.zip"), ROOT_DIR)
# shutil.copy(os.path.join(data_path,"0-301and302_400.zip"), ROOT_DIR)
# shutil.copy(os.path.join(data_path,"test 899_401.csv"), ROOT_DIR)
# shutil.copy(os.path.join(data_path,"900~1800.zip"), ROOT_DIR)
# shutil.copy(os.path.join(data_path,"test.csv"), ROOT_DIR)
# shutil.copy(os.path.join(data_path,"menu_w2v"), ROOT_DIR)
shutil.copy(os.path.join(data_path,"test_csv.csv"), ROOT_DIR)
shutil.copy(os.path.join(data_path,"include_db_menus.txt"), ROOT_DIR)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pexpect/popen_spawn.py:60: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  self._read_thread.setDaemon(True)


Mounted at /content/GoogleDrive/


'/content/include_db_menus.txt'

In [5]:
menu_csv = pd.read_csv(os.path.join(ROOT_DIR, "test_csv.csv"), encoding="euc-kr")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
print(len(menu_csv))
menu_csv = menu_csv.drop_duplicates()
print(len(menu_csv))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1912349
1372182


In [7]:
menus = []
unique_menus = []

for menu in menu_csv.menu2:
  try:
      split_list = menu.split(" ")
      fin_list = []
      for split_word in split_list:
        stop_matches = re.findall(f"[가-힣]*밥$", split_word)
        if len(stop_matches) > 0 and not ("알밥" in split_word or "볶음밥" in split_word or "덮밥" in split_word or "국밥" in split_word or "비빔밥" in split_word or "김밥" in split_word): continue
        stop_matches = re.findall(f"[가-힣]*김치$", split_word)
        if len(stop_matches) > 0: continue
        stop_matches = re.findall(f"[가-힣]*국$", split_word)
        if len(stop_matches) > 0 and not ( "해장국" in split_word): continue
        stop_matches = re.findall(f"[가-힣]*겉절이$", split_word)
        if len(stop_matches) > 0: continue
        stop_matches = re.findall(f"깍두기", split_word)
        if len(stop_matches) > 0: continue
        stop_matches = re.findall(f"과일샐러드", split_word)
        if len(stop_matches) > 0: continue
        stop_matches = re.findall(f"오이장아찌", split_word)
        if len(stop_matches) > 0: split_word = "피클"
        fin_list.append(split_word)
      if len(fin_list) >= 3:
        menus.append(fin_list)
        unique_menus.extend(fin_list)
  except:
    pass
unique_menus = list(set(unique_menus))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# for menu in menus:
#   if "안심스테이크" in " ".join(menu):
#     print(menu)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
te = TransactionEncoder()

te_ary = te.fit_transform(menus)

df = pd.DataFrame(te_ary, columns=te.columns_)

te = None
te_ary = None

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
frequent_itemsets = fpgrowth(df, min_support=1e-5, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=1e-5)
frequent_itemsets = None

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# 반찬 추천 함수 정의
def recommend_side_dish_confidence(input_dish, top_n=10):
  str_menu = " ".join(unique_menus)
  find_matches = re.findall(f"[가-힣]*{input_dish}[가-힣]*", str_menu)

  for menu_idx, menu in enumerate(find_matches):
    print("="*50)

    if menu not in list(rules['antecedents'].str.join(',')):
      print(f"입력한 메뉴({menu})가 학습 데이터에 없습니다.")
      continue

    print(f"입력한 메뉴({menu})의 반찬은 다음과 같습니다.")
    recommended_dishes = rules[rules['antecedents'].str.join(',') == menu].sort_values('confidence', ascending=False)[:top_n]
    print(recommended_dishes['consequents'].str.join(',').tolist())

def recommend_side_dish_conviction(input_dish, top_n=10):
  str_menu = " ".join(unique_menus)
  find_matches = re.findall(f"[가-힣]*{input_dish}[가-힣]*", str_menu)

  for menu_idx, menu in enumerate(find_matches):
    print("="*50)
    print(menu)

    if menu not in list(rules['antecedents'].str.join(',')):
      print(f"입력한 메뉴({menu})가 학습 데이터에 없습니다.")
      continue

    print(f"입력한 메뉴({menu})의 반찬은 다음과 같습니다.")
    recommended_dishes = rules[rules['antecedents'].str.join(',') == menu].sort_values('conviction', ascending=False)[:top_n]
    print(recommended_dishes['consequents'].str.join(',').tolist())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
input_dish = "치킨"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
recommend_side_dish_confidence(input_dish)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


입력한 메뉴(간장양념치킨)의 반찬은 다음과 같습니다.
['파무침', '치킨무', '김치볶음밥', '단무지', '김치찌개', '꽁치조림,파무침', '양상추샐러드', '부대찌개', '잔치국수', '라조기']
입력한 메뉴(치킨버거)의 반찬은 다음과 같습니다.
['감자튀김', '단무지', '볶음밥', '김치볶음밥', '우동', '양상추샐러드', '전복죽', '감자튀김,볶음밥', '감자튀김,양상추샐러드', '양배추샐러드']
입력한 메뉴(치킨너겟)의 반찬은 다음과 같습니다.
['떡볶이', '볶음밥', '단무지', '감자탕', '김치찌개', '콩나물무침', '부대찌개', '김치볶음밥', '비빔밥', '양상추샐러드']
입력한 메뉴(치킨데리야끼)가 학습 데이터에 없습니다.
입력한 메뉴(후라이드치킨)의 반찬은 다음과 같습니다.
['양배추샐러드', '치킨무', '단무지', '카레라이스', '떡볶이', '비빔밥', '김치찌개', '볶음밥', '콩나물무침', '부대찌개']
입력한 메뉴(치킨무)의 반찬은 다음과 같습니다.
['양배추샐러드', '후라이드치킨', '볶음밥', '떡볶이', '양념치킨', '닭강정', '감자튀김', '김치찌개', '부대찌개', '카레라이스']
입력한 메뉴(치킨까스)의 반찬은 다음과 같습니다.
['김치찌개', '부대찌개', '양상추샐러드', '양배추샐러드', '떡볶이', '단무지', '감자탕', '볶음밥', '콩나물무침', '육개장']
입력한 메뉴(양념치킨)의 반찬은 다음과 같습니다.
['단무지', '양배추샐러드', '볶음밥', '치킨무', '카레라이스', '잔치국수', '김치찌개', '무생채', '콩나물무침', '비빔밥']


In [14]:
recommend_side_dish_conviction(input_dish)

간장양념치킨
입력한 메뉴(간장양념치킨)의 반찬은 다음과 같습니다.
['치킨무', '파무침', '김치볶음밥', '꽁치조림,파무침', '꽁치조림', '라조기', '미나리나물', '단무지,김치볶음밥', '잔치국수', '동태찌개']
치킨버거


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


입력한 메뉴(치킨버거)의 반찬은 다음과 같습니다.
['감자튀김', '김치볶음밥', '전복죽', '우동', '감자튀김,볶음밥', '감자튀김,양상추샐러드', '감자튀김,김치볶음밥', '양상추샐러드', '감자튀김,전복죽', '감자튀김,우동']
치킨너겟
입력한 메뉴(치킨너겟)의 반찬은 다음과 같습니다.
['감자탕', '김치볶음밥', '카레라이스', '무조림', '김치찌개', '마파두부', '볶음우동', '김치볶음', '오이지무침', '제육덮밥']
치킨데리야끼
입력한 메뉴(치킨데리야끼)가 학습 데이터에 없습니다.
후라이드치킨
입력한 메뉴(후라이드치킨)의 반찬은 다음과 같습니다.
['치킨무', '양배추샐러드', '카레라이스', '비빔밥', '잔치국수', '오이부추무침', '치킨무,양배추샐러드', '김치볶음밥', '치킨무,떡볶이', '양념치킨']
치킨무
입력한 메뉴(치킨무)의 반찬은 다음과 같습니다.
['양배추샐러드', '후라이드치킨', '양념치킨', '닭강정', '감자튀김', '볶음밥', '볶음밥,양배추샐러드', '떡볶이,양배추샐러드', '닭튀김', '후라이드치킨,양배추샐러드']
치킨까스
입력한 메뉴(치킨까스)의 반찬은 다음과 같습니다.
['양배추샐러드', '양상추샐러드', '김치찌개', '부대찌개', '카레라이스', '감자탕', '육개장', '양송이스프', '두부조림', '어묵볶음']
양념치킨
입력한 메뉴(양념치킨)의 반찬은 다음과 같습니다.
['치킨무', '양배추샐러드', '카레라이스', '잔치국수', '무생채', '후라이드치킨', '콩비지찌개', '치킨무,양배추샐러드', '단무지무침,잔치국수', '볶음밥,양배추샐러드']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# rules.txt 다운(추가)

In [15]:
rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(부대찌개),(콩나물무침),0.047605,0.062405,0.003880,0.081502,1.306028,0.000909,1.020792,0.246032
1,(콩나물무침),(부대찌개),0.062405,0.047605,0.003880,0.062173,1.306028,0.000909,1.015534,0.249916
2,(부대찌개),(만두),0.047605,0.084039,0.002485,0.052207,0.621217,-0.001515,0.966414,-0.390326
3,(만두),(부대찌개),0.084039,0.047605,0.002485,0.029573,0.621217,-0.001515,0.981419,-0.399647
4,(부대찌개),(떡볶이),0.047605,0.112374,0.002807,0.058967,0.524739,-0.002542,0.943246,-0.487437


In [16]:
# rules.to_csv('/content/rules.csv', encoding='utf-8', index=False)
# shutil.copy(os.path.join(ROOT_DIR,"rules.csv"), data_path)
rules.to_csv('/content/rules.csv', encoding='utf-8', index=False)
shutil.copy(os.path.join(ROOT_DIR,"rules.csv"), data_path)

# with open('/content/rules.json', 'w', encoding='utf-8') as file:
#   json.dump(rules, file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'/content/GoogleDrive/MyDrive/rules.csv'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
